In [138]:
import import_ipynb
import Transformer
from CommonModule.Handle_Dir import mkdir_p, del_folder
from CommonModule.Embedding import DocumentEmbedding
from CommonModule.Encoder import IntegerEncoder


import re
import os
import numpy as np
from pathlib import Path
from glob import iglob
import tensorflow as tf
import sentencepiece as spm
import csv

In [71]:
BASE_DIR = "/docker/data/ksb/TestSampleDir"
DATA_BASE_DIR = os.path.join(BASE_DIR, 'articles')

PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")

SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Summary-Preprocessed-Data")

WORD_EMBEDDING_DIR = os.path.join(os.path.join(BASE_DIR, 'articleSummary-Jupyter'), 'Word-Embedding-Model')
MODEL_DIR = os.path.join(os.path.join(BASE_DIR, 'articleSummary-Jupyter'), 'Transformer-Model')

In [72]:
D_MODEL = 128

In [140]:
sp = spm.SentencePieceProcessor()
model_num = len(list(iglob('**.vocab', recursive=False))) -1
sp.Load('spm-{}.model'.format(model_num))

True

In [141]:
VOCAB_SIZE = 70000

In [142]:
options = {
    'model-type' : 'Sentence-Piece',
    'inv_wv' : None,
    'corpus' : None,
    'spm' : sp
}
input_encoded_list = IntegerEncoder(options=options, filepaths=list(iglob(os.path.join(PREPROCESSED_PATH, '**.csv'), recursive=False))).encoder()
output_encoded_list = IntegerEncoder(options=options, filepaths=list(iglob(os.path.join(SUMMARY_PREPROCESSED_PATH, '**.csv'), recursive=False))).encoder()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [143]:
LAYER_NUM = 6
NUM_HEADS = 8
DFF = 512

BATCH_SIZE = 64
BUFFER_SIZE = 5000

WARMUP_STEPS = 50
EPOCHS = 30

In [144]:
get_max_length = lambda x : np.max([len(line) for line in x])

MAX_LEN = get_max_length(input_encoded_list)
MAX_LEN

250

In [145]:
# Padding
input_encoded_matrix = tf.keras.preprocessing.sequence.pad_sequences(
    input_encoded_list, maxlen=MAX_LEN, padding='post')
summary_encoded_matrix = tf.keras.preprocessing.sequence.pad_sequences(
    output_encoded_list, maxlen=MAX_LEN, padding='post')

In [146]:
print('Contents Shape : {}'.format(input_encoded_matrix.shape))
print('Summaries Shape : {}'.format(summary_encoded_matrix.shape))

Contents Shape : (57632, 250)
Summaries Shape : (57632, 250)


In [147]:
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': input_encoded_matrix, # Encoder Input
        'dec_inputs': summary_encoded_matrix[:, :] # Decoder Input
    },
    {
        # Decoder Output, Remove <SOS>
        'outputs': summary_encoded_matrix[:, :]  
    },
))

In [148]:
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [149]:
for dict_1, dict_2 in dataset.as_numpy_iterator():
    print("encoder input : {enc}, decoder input : {dec}".format(enc= dict_1['inputs'].shape,
                                                               dec=dict_1['dec_inputs'].shape))
    print("output shape : {}".format(dict_2['outputs'].shape))

encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shape : (64, 250)
encoder input : (64, 250), decoder input : (64, 250)
output shap

In [150]:
lrate_scheduler = CustomSchedule(d_model=D_MODEL)

In [151]:
beta_1 = 0.9  
beta_2 = 0.98
epsilon = 10 ** -9

optimizer = tf.keras.optimizers.Adam(lrate_scheduler, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [152]:
model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=LAYER_NUM,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout = 0.3)

(1, 70000, 128)
(None, None, 128)
batch : Tensor("Placeholder_2:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder_2:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder_2:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder_2:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder_2:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder_2:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("Placeholder:0", shape=(None, None, 128), dtype=float32)
batch : Tensor("encoder/dropout_78/Identity:0", shape=(None, None, 128), dtype=float32)
ba

In [153]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LEN))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [ ]:
checkpoint_dirpath = os.path.join(MODEL_DIR, "Transformer")
checkpoint_dirpath = "Transformer"

mkdir_p(checkpoint_dirpath)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_dirpath,
    save_weights_only=True,
    monitor='loss',
    mode='max',
    save_best_only=True)

In [154]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

model.compile(optimizer=optimizer, loss=loss_function)
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 128)    10149632    inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [155]:
model.fit(dataset, batch_size=BATCH_SIZE, epochs=15, verbose=2, shuffle=True, callbacks=[model_checkpoint_callback])

Epoch 1/30
batch : Tensor("transformer/encoder/dropout_78/dropout/Mul_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/encoder/encoder_layer_0/layer_normalization_91/batchnorm/add_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/encoder/encoder_layer_1/layer_normalization_93/batchnorm/add_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/encoder/encoder_layer_2/layer_normalization_95/batchnorm/add_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/encoder/encoder_layer_3/layer_normalization_97/batchnorm/add_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/encoder/encoder_layer_4/layer_normalization_99/batchnorm/add_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/decoder/dropout_91/dropout/Mul_1:0", shape=(None, 250, 128), dtype=float32)
batch : Tensor("transformer/decoder/decoder_layer_0/layer_normalization_102/batchnorm/add_1:0", shape=(None, 25

ResourceExhaustedError: ignored